In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import kagglehub
import gc

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("aarishasifkhan/plantvillage-potato-disease-dataset")

print("Path to dataset files:", path)

100%|██████████| 37.8M/37.8M [00:00<00:00, 92.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/aarishasifkhan/plantvillage-potato-disease-dataset/versions/1


In [ ]:


BASE_MODEL_SAVE_PATH = 'efficientnet_stage1_wheat.h5'
NEW_MODEL_SAVE_PATH = 'efficientnet_stage2_potato.h5'
FINE_TUNE_LR = 1e-5
BATCH_SIZE = 32
VALIDATION_SPLIT_RATIO = 0.2
IMG_SIZE = (224, 224)

POTATO_ROOT_PATH = "/root/.cache/kagglehub/datasets/aarishasifkhan/plantvillage-potato-disease-dataset/versions/1"
POTATO_CLASS_DIR = os.path.join(POTATO_ROOT_PATH, 'PlantVillage')
N_Potato = 3 

print(f"Loading data from CORRECTED PATH: {POTATO_CLASS_DIR}")


def load_and_configure_dataset(data_dir, subset_name, split_ratio, shuffle_status=True):
    dataset = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        labels='inferred',
        label_mode='categorical',
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=shuffle_status,
        seed=42,
        validation_split=split_ratio,
        subset=subset_name,
    )

    def preprocess_efficientnet(image, label):
        image = tf.keras.applications.efficientnet.preprocess_input(image)
        return image, label

    dataset = dataset.map(preprocess_efficientnet, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset


potato_train_generator = load_and_configure_dataset(
    POTATO_CLASS_DIR, 'training', VALIDATION_SPLIT_RATIO
)
potato_validation_generator = load_and_configure_dataset(
    POTATO_CLASS_DIR, 'validation', VALIDATION_SPLIT_RATIO, shuffle_status=False
)

print(f"Number of classes for Potato (N_Potato): {N_Potato}")
print("Data loaded successfully using stable tf.data.Dataset.")

Loading data from CORRECTED PATH: /root/.cache/kagglehub/datasets/aarishasifkhan/plantvillage-potato-disease-dataset/versions/1/PlantVillage
Found 2152 files belonging to 3 classes.
Using 1722 files for training.
Found 2152 files belonging to 3 classes.
Using 430 files for validation.
Number of classes for Potato (N_Potato): 3
Data loaded successfully using stable tf.data.Dataset.


In [ ]:
import shutil

BASE_MODEL_SAVE_PATH = 'efficientnet_stage1_wheat .h5' 
N_Potato = 3 
def find_file(filename, search_path='/content'):
    for root, dirs, files in os.walk(search_path):
        if filename in files:
            return os.path.join(root, filename)
    return None

FULL_WEIGHTS_PATH = find_file(BASE_MODEL_SAVE_PATH)

if not FULL_WEIGHTS_PATH:
    print(f"FATAL ERROR: The weights file '{BASE_MODEL_SAVE_PATH}' was not found anywhere in the known directories.")
    print("Please confirm the file name is correct and the upload was successful.")
    raise FileNotFoundError(f"Missing required weights: {BASE_MODEL_SAVE_PATH}")

TARGET_PATH = os.path.join('/content/', BASE_MODEL_SAVE_PATH)
if FULL_WEIGHTS_PATH != TARGET_PATH:
    shutil.move(FULL_WEIGHTS_PATH, TARGET_PATH)
    FULL_WEIGHTS_PATH = TARGET_PATH 
print(f"✅ Weights file located and moved to: {FULL_WEIGHTS_PATH}")

✅ Weights file located and moved to: /content/efficientnet_stage1_wheat .h5


In [ ]:


wheat_model = tf.keras.models.load_model(FULL_WEIGHTS_PATH)

x = wheat_model.layers[-2].output 
output_potato = Dense(N_Potato, activation='softmax', name='potato_classification_head')(x)

model_potato = Model(inputs=wheat_model.input, outputs=output_potato)

model_potato.load_weights(NEW_MODEL_SAVE_PATH)
print(f"Successfully loaded best weights from '{NEW_MODEL_SAVE_PATH}' for Phase 2.")

for layer in model_potato.layers:
    layer.trainable = True

FINE_TUNE_LR_PHASE2 = 1e-6 

model_potato.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=FINE_TUNE_LR_PHASE2),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_potato.summary() 


checkpoint_phase2 = tf.keras.callbacks.ModelCheckpoint(
    filepath='efficientnet_stage2_potato_finetuned.weights.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    save_weights_only=True,
    verbose=1
)
early_stop_phase2 = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=7, 
    mode='max',
    verbose=1
)
reduce_lr_phase2 = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=4, 
    min_lr=1e-8,
    verbose=1
)

phase2_callbacks = [checkpoint_phase2, early_stop_phase2, reduce_lr_phase2]

epochs_phase2 = 20 
print(f"\n--- Starting Phase 2: Fine-Tuning with Unfrozen Layers (LR: {FINE_TUNE_LR_PHASE2}) ---")

history_potato_phase2 = model_potato.fit(
    potato_train_generator,
    validation_data=potato_validation_generator,
    epochs=epochs_phase2,
    callbacks=phase2_callbacks
)

print(f"\nPhase 2 fine-tuning complete. Best model weights saved as 'efficientnet_stage2_potato_finetuned.weights.h5'")

final_model_val_loss, final_model_val_acc = model_potato.evaluate(potato_validation_generator)
print(f"Final Model Validation Accuracy (after Phase 2): {final_model_val_acc:.4f}")

tf.keras.backend.clear_session()
gc.collect()


Successfully loaded best weights from 'efficientnet_stage2_potato.weights.h5' for Phase 2.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,053,414 (15.46 MB)

 Trainable params: 4,011,391 (15.30 MB)

 Non-trainable params: 42,023 (164.16 KB)


--- Starting Phase 2: Fine-Tuning with Unfrozen Layers (LR: 1e-06) ---
Epoch 1/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.3647 - loss: 1.1595 
Epoch 1: val_accuracy improved from -inf to 0.40465, saving model to efficientnet_stage2_potato_finetuned.weights.h5
54/54 ━━━━━━━━━━━━━━━━━━━━ 813s 14s/step - accuracy: 0.3646 - loss: 1.1595 - val_accuracy: 0.4047 - val_loss: 1.1486 - learning_rate: 1.0000e-06
Epoch 2/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.3568 - loss: 1.1606 
Epoch 2: val_accuracy improved from 0.40465 to 0.51860, saving model to efficientnet_stage2_potato_finetuned.weights.h5
54/54 ━━━━━━━━━━━━━━━━━━━━ 636s 12s/step - accuracy: 0.3570 - loss: 1.1603 - val_accuracy: 0.5186 - val_loss: 1.0445 - learning_rate: 1.0000e-06
Epoch 3/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.3789 - loss: 1.1148 
Epoch 3: val_accuracy improved from 0.51860 to 0.53953, saving model to efficientnet_stage2_potato_finetuned.weights.h5
54/54 ━━━━━━━━━━━━━━━━━━━━

0

**Reasoning**:
The previous Phase 2 fine-tuning only unfroze the last 30 layers, and the task requires unfreezing all layers of `model_potato` except the final classification head, setting a learning rate of `1e-7`, and training for 10 additional epochs. I will modify the unfreezing loop, learning rate, number of epochs, and checkpoint filename accordingly.

